## Usage of RNN LSTM to predict sales for the next month of different items across different shops on the basis of historical data of sales for last 34 months

This is the project for **Kaggle** competion. The data was provided by 1C company. <br>
Competition itself:  https://www.kaggle.com/c/competitive-data-science-predict-future-sales <br>
Total number of items: **21 800** <br>
Total number of sales: **2 935 849** <br>
Number of shops: **60**

The main idea is to create a RNN LSTM model for every item for every shop. 

### Data preprocessing

**Import libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

**Load datasets**

In [ ]:
df_sales = pd.read_csv('sales_train_v2.csv')
df_shops = pd.read_csv('shops.csv')
df_it_cat = pd.read_csv('item_categories.csv')
df_items = pd.read_csv('items.csv')

**Merge datasets**

In [ ]:
df_merged = df_sales.merge(df_shops,on='shop_id').merge(df_items,on='item_id')

**Drop uneeded columns**

In [ ]:
df_merged.drop(['date', 'shop_name', 'city', 'item_name','item_category_id'], axis=1, inplace=True

### Predict sales of each item per store

**In this example a price of a shop number 2 is predicted**

In [ ]:
from keras import backend as K
for kk in range(1):
    df_shop = df_merged.loc[df_merged['shop_id'] == 2]
    
    
    
    #pivot
    df_p = pd.pivot_table(df_shop,index=["date_block_num"],values=["item_cnt_day"], columns=["item_id"],aggfunc=[np.sum])
    
    #replace nan with 
    df_p = df_p.fillna(0)
    
    #######
    
    col_names = list(df_p.columns.values)
    col_list=[]
    for col in col_names:
        col_list.append(col[2])
    
    
    #layers, neurons, epochs, batch size
    #pred_list (4/50/100/32)
    #pred_list2(4/256/15/6) worse?
    #pred_list3(4/128/25/24) better?
    #pred_list4(4/128/25/40) better?
    #pred_list5(4/64/25/32) better?
    #pred_list6 (4/50/200/32)
    pred_list_zero=[]
    for ppr in range(2000,3000):   #range(len(col_names)):
        id_1 = df_p.iloc[:, ppr].values
        
        id_1 = id_1.reshape(-1, 1)
        
        # Feature Scaling
        from sklearn.preprocessing import MinMaxScaler
        sc = MinMaxScaler(feature_range = (0, 1))
        training_set_scaled = sc.fit_transform(id_1)
        
        X_train = []
        y_train = []
        for i in range(6, 34):
            X_train.append(training_set_scaled[i-6:i, 0])
            y_train.append(training_set_scaled[i, 0])
        X_train, y_train = np.array(X_train), np.array(y_train)
        
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        
        
        
        
        # Initialising the RNN
        regressor = Sequential()
        
        # Adding the first LSTM layer and some Dropout regularisation
        #units - cells
        #return_sequences - stacked lstm layers
        regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
        regressor.add(Dropout(0.2))
        
        # Adding a second LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = 50, return_sequences = True))
        regressor.add(Dropout(0.2))
        
        # Adding a third LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = 50, return_sequences = True))
        regressor.add(Dropout(0.2))
        
        # Adding a fourth LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = 50))
        regressor.add(Dropout(0.2))
        
        # Adding the output layer
        regressor.add(Dense(units = 1))
        
        # Compiling the RNN
        regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
        
        # Fitting the RNN to the Training set
        regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
        
        
        
        X_test = []
        for i in range(33, 34): 
            X_test.append(training_set_scaled[i-6:i, 0])
        X_test = np.array(X_test)
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        
        predicted_sales = regressor.predict(X_test)
        predicted_sales_reverse = sc.inverse_transform(predicted_sales) #return back
        pred_list_zero.append(predicted_sales_reverse[0,0])
        #item_id = col_names[ppr][2]
        
        #dict_id_sales[item_id] = predicted_sales_reverse
        #dict_id_shops[kk] = dict_id_sales
        
        
        
        
        print("Id ",ppr," out of ", len(col_names))
        print("##/##/##/##/##/##/")
        K.clear_session()